In [5]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)
con.sql("DROP TABLE IF EXISTS 재고;")
con.sql("""
CREATE TABLE 재고 (
    재고ID INTEGER PRIMARY KEY,
    창고명 VARCHAR,
    품목코드 VARCHAR,
    재고수량 INTEGER,
    출고 INTEGER,
    평가액 INTEGER
);
""")
con.sql("""
INSERT INTO 재고 (재고ID, 창고명, 품목코드, 재고수량, 출고, 평가액) VALUES
(501, '부산', 'P100', 500, 50, 2500000),
(502, '부산', 'P200', 100, 80, 500000),
(503, '인천', 'P100', 600, 30, 3000000),
(504, '인천', 'P300', 800, 150, 4000000),
(505, '대구', 'P200', 200, 10, 1000000),
(506, '대구', 'P400', 400, 120, 2000000),
(507, '부산', 'P300', 1000, 20, 5000000);
""")
print("--- 1. [재고] 테이블 데이터 확인 ---")
con.sql("SELECT * FROM 재고;").show()

--- 1. [재고] 테이블 데이터 확인 ---
┌────────┬─────────┬──────────┬──────────┬───────┬─────────┐
│ 재고ID │ 창고명  │ 품목코드 │ 재고수량 │ 출고  │ 평가액  │
│ int32  │ varchar │ varchar  │  int32   │ int32 │  int32  │
├────────┼─────────┼──────────┼──────────┼───────┼─────────┤
│    501 │ 부산    │ P100     │      500 │    50 │ 2500000 │
│    502 │ 부산    │ P200     │      100 │    80 │  500000 │
│    503 │ 인천    │ P100     │      600 │    30 │ 3000000 │
│    504 │ 인천    │ P300     │      800 │   150 │ 4000000 │
│    505 │ 대구    │ P200     │      200 │    10 │ 1000000 │
│    506 │ 대구    │ P400     │      400 │   120 │ 2000000 │
│    507 │ 부산    │ P300     │     1000 │    20 │ 5000000 │
└────────┴─────────┴──────────┴──────────┴───────┴─────────┘



In [ ]:
# "GROUP BY 창고명"에 의해서 나누어진 그룹

# 재고ID,    창고명,     품목코드,      재고수량,     출고,       평가액
# 501,      부산,       P100,         500,        50,         2500000
# 502,      부산,       P200,         100,        80,         500000
# 507,      부산,       P300,         1000,       20,         5000000

# 재고ID,    창고명,     품목코드,      재고수량,     출고,       평가액
# 503,      인천,       P100,         600,        30,         3000000
# 504,      인천,       P300,         800,        150,        4000000

# 재고ID,    창고명,     품목코드,      재고수량,     출고,       평가액
# 505,      대구,       P200,         200,        10,         1000000
# 506,      대구,       P400,         400,        120,        2000000

# SELECT
#     창고명,
#     SUM(재고수량) AS 총_재고,
#     SUM(출고) AS 총_출고,
#     SUM(평가액) AS 총_평가액,
#     
#    SUM(CASE WHEN 재고수량 > 500 THEN 1 ELSE 0 END) AS 고재고_품목수
# 을 실행

# 창고명    총_재고     총_출고  총_평가액  고재고_품목수 
# 부산        1600      150    8000000       1 
# 인천        1400      180    7000000       2 
# 대구         600      130    3000000       0

In [6]:
print("\n--- 2. 비효율 창고 (수정된 HAVING 조건 적용) ---")
con.sql("""
SELECT
    창고명,
    SUM(재고수량) AS 총_재고,
    SUM(출고) AS 총_출고,
    SUM(평가액) AS 총_평가액,
    
    SUM(CASE WHEN 재고수량 > 500 THEN 1 ELSE 0 END) AS 고재고_품목수
FROM
    재고
GROUP BY
    창고명
-- 수정: 총 출고 < 100 -> 총 출고 < 200 으로 완화
HAVING
    총_평가액 >= 5000000 AND 총_출고 < 200 
ORDER BY
    총_재고 DESC
LIMIT
    2;
""").show()

con.close()


--- 2. 비효율 창고 (수정된 HAVING 조건 적용) ---
┌─────────┬─────────┬─────────┬───────────┬───────────────┐
│ 창고명  │ 총_재고 │ 총_출고 │ 총_평가액 │ 고재고_품목수 │
│ varchar │ int128  │ int128  │  int128   │    int128     │
├─────────┼─────────┼─────────┼───────────┼───────────────┤
│ 부산    │    1600 │     150 │   8000000 │             1 │
│ 인천    │    1400 │     180 │   7000000 │             2 │
└─────────┴─────────┴─────────┴───────────┴───────────────┘



In [ ]:
# With - 1

In [7]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

con.sql("DROP TABLE IF EXISTS Project;")
con.sql("""
CREATE TABLE Project (기록ID INTEGER, 팀명 VARCHAR, 프로젝트명 VARCHAR, 투입시간 INTEGER, 총비용 INTEGER);
INSERT INTO Project (기록ID, 팀명, 프로젝트명, 투입시간, 총비용) VALUES
(1, '알파팀', '웹사이트 개발', 40, 2000000), 
(2, '베타팀', '모바일 앱 개발', 60, 3600000), 
(3, '알파팀', '데이터 분석', 20, 1100000), 
(4, '감마팀', '웹사이트 개발', 50, 2000000), 
(5, '베타팀', '모바일 앱 개발', 30, 1800000), 
(6, '알파팀', '웹사이트 개발', 10, 500000), 
(7, '감마팀', '데이터 분석', 70, 3150000), 
(8, '베타팀', '데이터 분석', 40, 2600000);
""")
print("--- Project 테이블 데이터 확인 ---")
con.sql("SELECT 팀명, 총비용 FROM Project;").show()

--- Project 테이블 데이터 확인 ---
┌─────────┬─────────┐
│  팀명   │ 총비용  │
│ varchar │  int32  │
├─────────┼─────────┤
│ 알파팀  │ 2000000 │
│ 베타팀  │ 3600000 │
│ 알파팀  │ 1100000 │
│ 감마팀  │ 2000000 │
│ 베타팀  │ 1800000 │
│ 알파팀  │  500000 │
│ 감마팀  │ 3150000 │
│ 베타팀  │ 2600000 │
└─────────┴─────────┘



In [ ]:
con.sql("""
WITH TeamSummary AS (
    -- 1단계: 팀별 총비용을 계산하여 'TeamSummary'라는 임시 테이블로 정의
    SELECT
        팀명,
        SUM(총비용) AS 총비용
    FROM
        Project
    GROUP BY
        팀명
)
-- 2단계: 임시 테이블 'TeamSummary'를 FROM 절에서 사용하여 필터링
SELECT
    팀명,
    총비용
FROM
    TeamSummary
WHERE
    총비용 >= 5000000;
""").show()

con.close()

In [ ]:
# With - 2

In [9]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

# 오류 방지 및 데이터 재설정
con.sql("DROP TABLE IF EXISTS 직원;")
con.sql("""
CREATE TABLE 직원 (
    직원ID INTEGER, 
    부서명 VARCHAR, 
    급여 INTEGER
);
""")
con.sql("""
INSERT INTO 직원 (직원ID, 부서명, 급여) VALUES
(101, '인사팀', 6000), 
(102, '인사팀', 4000), 
(103, '개발팀', 8000), 
(104, '개발팀', 7000), 
(105, '개발팀', 9000); 
-- 전체 평균: (6000+4000+8000+7000+9000) / 5 = 6800
-- 인사팀 평균: 5000 (전체 평균보다 낮음)
-- 개발팀 평균: 8000 (전체 평균보다 높음)
""")
print("--- 직원 테이블 데이터 확인 ---")
con.sql("SELECT * FROM 직원;").show()

--- 직원 테이블 데이터 확인 ---
┌────────┬─────────┬───────┐
│ 직원ID │ 부서명  │ 급여  │
│ int32  │ varchar │ int32 │
├────────┼─────────┼───────┤
│    101 │ 인사팀  │  6000 │
│    102 │ 인사팀  │  4000 │
│    103 │ 개발팀  │  8000 │
│    104 │ 개발팀  │  7000 │
│    105 │ 개발팀  │  9000 │
└────────┴─────────┴───────┘



In [10]:
print("\n--- 2. WITH와 서브쿼리를 이용한 순수 비교 분석 (JOIN 없음) ---")
con.sql("""
WITH DeptAvg AS (
    -- 1단계: 각 부서별 평균 급여를 계산하여 'DeptAvg' 임시 테이블로 저장
    SELECT
        부서명,
        AVG(급여) AS 부서평균
    FROM
        직원
    GROUP BY
        부서명
)
-- 2단계: 임시 테이블 'DeptAvg'를 사용하고, WHERE 절에서 (전체 평균) 서브쿼리와 비교
SELECT
    부서명,
    부서평균
FROM
    DeptAvg
WHERE
    -- **핵심:** 부서평균이 (전체 직원의 평균 급여)보다 큰 그룹만 필터링
    부서평균 > (SELECT AVG(급여) FROM 직원);
""").show()

con.close()


--- 2. WITH와 서브쿼리를 이용한 순수 비교 분석 (JOIN 없음) ---
┌─────────┬──────────┐
│ 부서명  │ 부서평균 │
│ varchar │  double  │
├─────────┼──────────┤
│ 개발팀  │   8000.0 │
└─────────┴──────────┘



In [ ]:
# with + group by

In [11]:
import duckdb
con = duckdb.connect(database=':memory:', read_only=False)

# 오류 방지 및 데이터 재설정
con.sql("DROP TABLE IF EXISTS 직원;")
con.sql("""
CREATE TABLE 직원 (
    직원ID INTEGER, 
    부서명 VARCHAR, 
    성과점수 INTEGER -- 100점 만점, 60점 이하를 저성과자로 간주
);
""")
con.sql("""
INSERT INTO 직원 (직원ID, 부서명, 성과점수) VALUES
(101, '마케팅', 90), 
(102, '마케팅', 55), 
(103, '개발', 85), 
(104, '개발', 70), 
(105, '개발', 60),
(106, '인사', 50),
(107, '인사', 95);
""")
print("--- 직원 테이블 데이터 확인 ---")
con.sql("SELECT * FROM 직원;").show()

--- 직원 테이블 데이터 확인 ---
┌────────┬─────────┬──────────┐
│ 직원ID │ 부서명  │ 성과점수 │
│ int32  │ varchar │  int32   │
├────────┼─────────┼──────────┤
│    101 │ 마케팅  │       90 │
│    102 │ 마케팅  │       55 │
│    103 │ 개발    │       85 │
│    104 │ 개발    │       70 │
│    105 │ 개발    │       60 │
│    106 │ 인사    │       50 │
│    107 │ 인사    │       95 │
└────────┴─────────┴──────────┘



In [12]:
print("\n--- 2. CTE와 CASE WHEN을 활용한 저성과자 비율 분석 ---")
con.sql("""
WITH DeptPerformance AS (
    -- 1단계: 조건부 집계 (CASE WHEN)를 사용하여 저성과자 수와 총 직원 수를 계산
    SELECT
        부서명,
        COUNT(직원ID) AS 총직원수,
        -- 성과점수 60점 이하를 저성과자로 분류하여 COUNT
        SUM(CASE WHEN 성과점수 <= 60 THEN 1 ELSE 0 END) AS 저성과자수
    FROM
        직원
    GROUP BY
        부서명
)
-- 2단계: CTE의 결과를 활용하여 비율을 계산하고 순위 매기기
SELECT
    부서명,
    총직원수,
    저성과자수,
    -- 비율 계산 (저성과자수 / 총직원수)
    ROUND(
        (저성과자수 * 100.0) / 총직원수,
        2
    ) AS 저성과자_비율_퍼센트
FROM
    DeptPerformance
ORDER BY
    저성과자_비율_퍼센트 DESC; -- 비율이 높은 부서부터 정렬
""").show()

con.close()


--- 2. CTE와 CASE WHEN을 활용한 저성과자 비율 분석 ---
┌─────────┬──────────┬────────────┬──────────────────────┐
│ 부서명  │ 총직원수 │ 저성과자수 │ 저성과자_비율_퍼센트 │
│ varchar │  int64   │   int128   │        double        │
├─────────┼──────────┼────────────┼──────────────────────┤
│ 마케팅  │        2 │          1 │                 50.0 │
│ 인사    │        2 │          1 │                 50.0 │
│ 개발    │        3 │          1 │                33.33 │
└─────────┴──────────┴────────────┴──────────────────────┘

